<a href="https://colab.research.google.com/github/mroopeshunt/Densho-Digital-Repo-Analysis/blob/main/Topic%20Modelling/Bertopic_TopicModelling_unigrams%26bigrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
#!pip install stop_words

In [54]:
#!pip install bertopic

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import json
import os
import glob
import pprint

 
count = 0
answers_dict = {'narr_answers': []}
#path = '/content/gdrive/Narrators Json Files-With Q&A/'
#for filename in glob.glob(os.path.join(path, '*.json')): #only process .JSON files in folder.
for filename in glob.glob('/content/gdrive/My Drive/Narrators Json Files-With Q&A/*.json'):     
    #print(filename)
    narrators_answers_corpus = ""
    with open(filename, mode = 'rb') as currentFile:
        data = json.loads(currentFile.read())
        try:
            if data['transcript'] != {}:
                for segment in data["transcript"].keys():
                    for question in data["transcript"][segment]['Segment Q&A'].keys():
                        narrators_answers_corpus = narrators_answers_corpus + " " + data["transcript"][segment]['Segment Q&A'][question]
                        count = count + 1                
            else:
                pass
        except Exception as e:
            print(e)
            
    answers_dict['narr_answers'].append(narrators_answers_corpus) 

print()


In [5]:
import pandas as pd
df = pd.DataFrame(answers_dict)
df['narr_answers'] = df['narr_answers'].str.lower()
print(df['narr_answers'].shape)

(1042,)


In [6]:
df['narr_answers'] = df['narr_answers'].str.replace('[^a-zA-Z0-9 ]', '')

import re
from stop_words import get_stop_words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
stop_words = list(get_stop_words('en'))

print((set(list(STOPWORDS)) | set(list(stop_words))))
stopwords = list(set(list(STOPWORDS)) | set(list(stop_words)))
print(type(list(STOPWORDS)))
print(type((stop_words)))
stopwords.extend(['oh', 'yeah','like', 'came', 'back', 'go', 'know', 'said', 'went', 'us', 'think', 'one'])
print(len(stopwords))
stopwords = [re.sub('[^a-zA-Z0-9]+', '', word) for word in stopwords]
df['clean_narr_answers'] = df['narr_answers'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'ve', 'hasn', 'her', 't', "i'll", 's', 'mustn', "couldn't", "you'll", 'would', 'wouldn', 'does', 'few', 'what', "that's", 'into', 'than', "they're", 'own', 'now', "shouldn't", 'between', 'weren', 'yourselves', 'wasn', "you're", 'as', 'all', 'only', 'won', 'their', 'is', 'some', 'by', 'which', 'isn', "he's", 'do', 'shouldn', 'each', 'o', "it's", 'ain', "we've", 'up', 'hadn', "hasn't", 'm', "she'd", 'my', 'where', 'him', 'being', 'who', "they'd", "mightn't", "we'll", "here's", 'ma', "we're", 'just', 'with', 'out', 'from', 'cannot', 'at', 'whom', "haven't", "that'll", 'an', 'how', 'very', 'over', "who's", 're', 'myself', "don't", 'to', 'it', 'any', 'no', 'ourselves', 'after', "they've", 'he', 'can', 'd', "they'll", 'couldn', "doesn't", "why's", 'be', 'once', 'so', 'theirs', 'under', "let's", 'been', "what's", 'them', "should've", 'these', 'here', "hadn't", 'having', 'she', 'did'

In [8]:
print(len(list(df['clean_narr_answers'])))
doc = []
for i in list(df['clean_narr_answers']):
  if i != "":
    doc.append(i)

print(len(doc))

1042
945


In [9]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(doc)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

2022-03-04 05:22:05,213 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-03-04 05:22:18,363 - BERTopic - Reduced dimensionality with UMAP
2022-03-04 05:22:18,460 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [13]:
freq = topic_model.get_topic_info(); freq.head(15)

,Topic,Count,Name
0,-1,491,-1_people_got_time_japanese
1,0,88,0_school_got_japanese_people
2,1,54,1_got_school_japanese_time
3,2,45,2_people_japanese_got_time
4,3,40,3_school_japanese_got_time
5,4,38,4_remember_got_people_school
6,5,38,5_japanese_time_people_got
7,6,30,6_japanese_war_army_japan
8,7,25,7_jacl_people_government_japanese
9,8,23,8_got_time_get_used


In [14]:
topic_model.get_topic(1)  # Select the most frequent topic

[('got', 0.019862443158102733),
 ('school', 0.01862532194456568),
 ('japanese', 0.01859545674507755),
 ('time', 0.018105674535022365),
 ('people', 0.01765387145252257),
 ('get', 0.016509515309625324),
 ('remember', 0.01609906129670022),
 ('father', 0.015302561963305648),
 ('used', 0.015043054018393646),
 ('yes', 0.014658263893137647)]

In [15]:
topic_model.visualize_topics()

In [16]:
topic_model.visualize_distribution(probs[200], min_probability=0.000)

In [22]:
topic_model.visualize_hierarchy(top_n_topics=15)

In [25]:
topic_model.visualize_barchart(top_n_topics=15, n_words = 10)

In [27]:
topic_model.visualize_heatmap(n_clusters=14, width=1000, height=1000)

In [28]:
topic_model.visualize_term_rank()

In [29]:
topic_model.update_topics(doc, topics, n_gram_range=(2, 2))

In [30]:
topic_model.get_topic(10)   # We select topic that we viewed before

[('jpn jpn', 0.014590772991220443),
 ('jpn yes', 0.013082637723497935),
 ('yes jpn', 0.009171931273097086),
 ('eng uhhuh', 0.006187188450201397),
 ('jpn right', 0.006015696852183141),
 ('uhhuh eng', 0.005843488832611496),
 ('uhhuh jpn', 0.005670540494386049),
 ('jpn see', 0.005670540494386049),
 ('eng eng', 0.005496826291381605),
 ('yes uhhuh', 0.005406359197255124)]

In [40]:
freq = topic_model.get_topic_info(); freq.head(20)

,Topic,Count,Name
0,-1,491,-1_high school_little bit_united states_los an...
1,0,88,0_high school_san francisco_little bit_japanes...
2,1,54,1_high school_little bit_terminal island_years...
3,2,45,2_japanese americans_pearl harbor_united state...
4,3,40,3_high school_hood river_united states_assembl...
5,4,38,4_high school_little bit_pearl harbor_got married
6,5,38,5_high school_united states_san francisco_tule...
7,6,30,6_united states_camp savage_japanese language_...
8,7,25,7_japanese americans_japanese american_supreme...
9,8,23,8_everything else_high school_yes uhhuh_tule lake


In [32]:
topic_model.visualize_topics()

In [33]:
topic_model.visualize_distribution(probs[200], min_probability=0.000)

In [43]:
topic_model.visualize_hierarchy(top_n_topics=14)

In [51]:
topic_model.visualize_barchart(top_n_topics=14, n_words = 10, width = 350, height = 250)

In [52]:
topic_model.visualize_heatmap(n_clusters=14, width=1000, height=1000)

In [55]:
topic_model.visualize_term_rank()